Christian Sutton  
10/17/20      
Module 7  

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
#import matplotlib.pyplot as plt
SRdata = pd.read_csv("C:/Users/physi/Desktop/AppliedMachineLearning_EN.705.601/Mod3/master.csv")

In [2]:
#I am removing countries that have been contributing repeated zeros to the data set

SRdata = SRdata[SRdata.country != "Antigua and Barbuda"]
SRdata = SRdata[SRdata.country != "Aruba"]
SRdata = SRdata[SRdata.country != "Bosnia and Herzegovina"]
SRdata = SRdata[SRdata.country != "Cabo Verdea"]
SRdata = SRdata[SRdata.country != "Cyprus"]
SRdata = SRdata[SRdata.country != "Dominica"]
SRdata = SRdata[SRdata.country != "Grenada"]
SRdata = SRdata[SRdata.country != "Jamaica"]
SRdata = SRdata[SRdata.country != "Kuwait"]
SRdata = SRdata[SRdata.country != "Maldives"]
SRdata = SRdata[SRdata.country != "Montenegro"]
SRdata = SRdata[SRdata.country != "Oman"]
SRdata = SRdata[SRdata.country != "Qatar"]
SRdata = SRdata[SRdata.country != "Saint Kitts and Nevis"]

In [3]:
#Since HDI for year -  is missing most of it's values, this feature will be removed
#Country-year -  is redundant to features "year" and "country"
#gdp_for_year - removing this feature because it isn't used later
#removing counntry, year and gdp_per_capita, due to low correlation
#population and suicides_no are being removed due to unwanted correlation between features

SRdata =SRdata.drop(["HDI for year","country-year"," gdp_for_year ($) ","country","year",\
                     "gdp_per_capita ($)","population","suicides_no"], axis =1)

In [4]:
SRdata.head()

,sex,age,suicides/100k pop,generation
0,male,15-24 years,6.71,Generation X
1,male,35-54 years,5.19,Silent
2,female,15-24 years,4.83,Generation X
3,male,75+ years,4.59,G.I. Generation
4,male,25-34 years,3.28,Boomers


In [5]:
# OneHOTEncode for problem 1 fitting 
SRdata1 = pd.get_dummies(SRdata)
SRdata1 =SRdata1.drop(["sex_female","age_75+ years","generation_Silent"], axis =1)
SRdata1.head()

,suicides/100k pop,sex_male,age_15-24 years,age_25-34 years,age_35-54 years,age_5-14 years,age_55-74 years,generation_Boomers,generation_G.I. Generation,generation_Generation X,generation_Generation Z,generation_Millenials
0,6.71,1,1,0,0,0,0,0,0,1,0,0
1,5.19,1,0,0,1,0,0,0,0,0,0,0
2,4.83,0,1,0,0,0,0,0,0,1,0,0
3,4.59,1,0,0,0,0,0,0,1,0,0,0
4,3.28,1,0,1,0,0,0,1,0,0,0,0


In [6]:
# copy dataset and use for MAE y-true, y-predict, 1002 Rows
SRFiltered = SRdata1

#filter for Male, 20, Gen X
SRFiltered = SRFiltered[SRFiltered.sex_male != 0]
SRFiltered = SRFiltered[SRFiltered['age_15-24 years'] != 0] 
SRFiltered = SRFiltered[SRFiltered['generation_Generation X'] != 0] 
SRFiltered.head()

,suicides/100k pop,sex_male,age_15-24 years,age_25-34 years,age_35-54 years,age_5-14 years,age_55-74 years,generation_Boomers,generation_G.I. Generation,generation_Generation X,generation_Generation Z,generation_Millenials
0,6.71,1,1,0,0,0,0,0,0,1,0,0
13,5.33,1,1,0,0,0,0,0,0,1,0,0
28,3.71,1,1,0,0,0,0,0,0,1,0,0
37,3.41,1,1,0,0,0,0,0,0,1,0,0
48,7.40,1,1,0,0,0,0,0,0,1,0,0


# Problem 1
**[20 pts] Keep the variables as one-hot encoded and develop a multiple linear regression model. Use your model to predict the target variable for the people with age 20, male, and generation X. What is the MAE error of this prediction? How many line coefficients are there?**  



In [7]:
from sklearn import linear_model
reg = linear_model.LinearRegression().fit(SRdata1.iloc[:,1:], SRdata1.iloc[:,0])

# coefficients for oneHotEncoded features. 11 coefficients for the 11 column features I'm fitting 
reg.coef_

array([ 15.77651065, -13.5495341 , -10.70817809,  -8.30171004,
       -22.20322163,  -7.4834319 ,   0.07376695,   2.55850094,
        -0.69344613,  -1.37886703,  -1.59621382])

In [8]:
reg.intercept_

16.378536153322425

In [9]:
Y_Pred = reg.predict(SRFiltered.iloc[:,1:])

# MAE ERROR Male, 20yrs, Gen X
from sklearn.metrics import mean_absolute_error
mean_absolute_error(SRFiltered.iloc[:,0], Y_Pred)

9.443522385685892

In [10]:
# prediction 20 yr, male , gen x
reg.predict(np.array([1,1,0,0,0,0,0,0,1,0,0]).reshape(1,-1))

array([17.91206657])

# Problem 2
**[30 pts] Now use the original sex, age and generation variables in numerical form and develop a new model. Use your model to predict the target value for the people with age 20, male, and generation X. What is the MAE error of this prediction? How many line coefficients are there? (Note that for this step you have to think of a way of encoding the original nominal age feature and generation feature into numerical features.)**  



In [11]:
# encode values
ages = {'15-24 years':20, '5-14 years':10,'25-34 years':30,'35-54 years':45,'55-74 years':65,'75+ years':75}
Generations = {'G.I. Generation':1,'Silent':2,'Boomers':3,\
               'Generation X':4,'Millenials':5,'Generation Z':6}
sex ={'male':1, 'female':0}

SRdata['age']= SRdata['age'].map(ages)
SRdata['generation']= SRdata['generation'].map(Generations)
SRdata['sex']=SRdata['sex'].map(sex)

In [12]:
# create data for problem two 
SRdata2 = SRdata
SRdata2.head()

,sex,age,suicides/100k pop,generation
0,1,20,6.71,4
1,1,45,5.19,2
2,0,20,4.83,4
3,1,75,4.59,1
4,1,30,3.28,3


In [13]:
# MAE data y-predict, y-true 20yr, male, Genx
SRFilter2 = SRdata
SRFilter2 = SRFilter2[SRFilter2.sex != 0]
SRFilter2 = SRFilter2[SRFilter2.age == 20] 
SRFilter2 = SRFilter2[SRFilter2.generation == 4] 
SRFilter2.head()

,sex,age,suicides/100k pop,generation
0,1,20,6.71,4
13,1,20,5.33,4
28,1,20,3.71,4
37,1,20,3.41,4
48,1,20,7.40,4


In [14]:
reg2 = linear_model.LinearRegression().fit(SRdata2.iloc[:,[0,1,3]], SRdata2.iloc[:,2])

# Coefficients for numeric features. 3 coefficients for the 3 columns features I'm fitting 
reg2.coef_

array([15.77651065,  0.24052537, -1.12001762])

In [15]:
reg2.intercept_

-0.32558987282572005

In [16]:
#make prediction for problem 2
Y_Pred2 = reg2.predict(SRFilter2.iloc[:,[0,1,3]])

# MAE ERROR of 20y , male , genX
mean_absolute_error(SRFilter2.iloc[:,2], Y_Pred2)

8.74733190009631

In [17]:
# prediction 20yr, male, gen x
reg2.predict(np.array([1, 20, 4]).reshape(1,-1))

array([15.78135764])

# Problem 3
**[10 pts] Did you note any change in these two model performances?**  

The first model from problem 1 had a higher MAE and higher rate of suicide. The second model had both a lower MAE and lower rate of suicide. 


# Problem 4

** [10 pts] What is the prediction for age 33, male and generation Alpha (i.e. the generation after generation Z)?**  



In [18]:
# prediction for male=1 , age 33 = 30, gen alpha =7 
reg2.predict(np.array([1, 30, 7]).reshape(1,-1))

array([14.82655845])

# Problem 5
**[10 pts] Give one advantage when using regression (as opposed to classification with nominal features) in terms of input data features.**  

Regression allows the data scientist to build a model that targets prediction for any crosssection of the the model. I could quickly check the risk of suicide for any combination of features I wish. In classification, I built a whole module for just one type of prediction, such as high risk or not

# Problem 6 

**[10 pts bonus] Give one advantage when using regular numerical values rather than one-hot encoding for regression.**  

One advantage is there are less features and the data is represented compactly. In this case the numerical data had less error 



# Problem 7

** [10 pts bonus] Now that you developed both a classifier and a regression model for this problem, which method do you suggest to your machine learning model customer and why?**  

In my opinion I like the regression model for ease of use and for it's more simplistic explaination of how the model is constructed. The regression model more intuitive and eaiser to describe to a customer versus a classification model on a random forest classifer. 